In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import time

In [2]:
# actions = ['come', 'away', 'spin', 'stop']
actions = ['Come', 'Away', 'Spin', 'Stop', 'x1', 'x2', 'x3']
seq_length = 30
date = '0518_1_1'
model = load_model('models/{}/model.keras'.format(date))

In [3]:
# def on_connect(client, mosq, obj, rc):
#     print("on_connect:: Connected with result code " + str(rc))
#     print("rc: " + str(rc))

# def on_message(mosq, obj, msg):
#     topic = msg.topic
#     payload = msg.payload.decode("utf-8")
#     a = topic + payload
#     print(a)

# def on_publish(client, obj, mid):
#     print("mid: ", str(mid), "obj: ", actions)

# def on_log(mosq, obj, level, string):
#     print(string)

# RST = 24
# DC = 23
# SPI_PORT = 0
# SPI_DEVICE = 0

# client = mqtt.Client()
# client.on_message = on_message
# client.on_connect = on_connect
# client.on_publish = on_publish

# client.on_log = on_log

# # 학교 601 와이파이
# # client.connect('192.168.0.3', 1883, 60)

# # 핫스팟
# # client.connect('192.168.116.74', 1883, 60)

# # 집 와이파이
# # client.connect('192.168.219.105', 1883, 60)
# client.loop_start()

In [4]:
# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

# w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# out = cv2.VideoWriter('input.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))
# out2 = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))

seq = []
action_seq = []

while cap.isOpened():
    ret, img = cap.read()
    img0 = img.copy()

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
            v = v2 - v1 # [20, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree

            d = np.concatenate([joint.flatten(), angle])

            seq.append(d)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) < seq_length:
                continue

            input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)

            y_pred = model.predict(input_data).squeeze()

            i_pred = int(np.argmax(y_pred))
            conf = y_pred[i_pred]

            # 인식률
            # 정확도가 conf보다 클 경우 해당 동작으로 인식
            if conf < 0.95:
                continue

            action = actions[i_pred]
            action_seq.append(action)
            # 3번 연속으로 인식될 경우 해당 동작으로 판별
            if len(action_seq) < 3:
                continue

            this_action = ""
            if action_seq[-1] == action_seq[-2] == action_seq[-3]:# == action_seq[-4] == action_seq[-5]:# == action_seq[-6] == action_seq[-7] == action_seq[-8] == action_seq[-9] == action_seq[-10]:
                this_action = action

            cv2.putText(img, f'{this_action.upper()}', org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 0, 0), thickness=2)
            #client.publish("hand_gesture", str(this_action))
            print(this_action)
            

    # out.write(img0)
    # out2.write(img)
    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        print("exiting...")
        break

# Release video capture and close windowsq
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Stop
1/1 ━━━━━━━━━━━━━━━━